In [1]:
from jupyterirtools import sso
import os
import time
import importlib
import json

sso.login("Jupyter-IR-AdministratorAccess", os.environ['LOGGING_ACCOUNT'])

If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=SLXV-DWWV to activate the session


<IPython.core.display.Javascript object>

In [27]:
import boto3
session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['LOGGING_ACCOUNT'])
ssm_client = session.client('ssm')
aggregator_param_response = ssm_client.get_parameter(Name='Jupyter-CloudTrail-Destination')

session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])
org_client = session.client('organizations')
desc_client = org_client.describe_organization()
org_unit_id = org_client.list_roots()['Roots'][0]['Id']


cloudtrail_logs_location = f"{aggregator_param_response['Parameter']['Value']}AWSLogs/{desc_client['Organization']['Id']}"

ec2_client = session.client('ec2')
regions = []

region_response = ec2_client.describe_regions()

for region in region_response['Regions']:
    regions.append(region['RegionName'])
    
flow_log_response = ec2_client.describe_flow_logs()

flow_logs_path = ""
flow_logs_table_name = ""
flow_logs_type = "plain-text"

if len(flow_log_response['FlowLogs']) == 0:
    no_flow_logs.append(f"{vpc['VpcId']:20}\t{account}\t{region_name}")
else:
    flow_log_info = flow_log_response['FlowLogs'][0]
    print(flow_log_info)
    if flow_log_info['LogDestinationType'] == "s3":
        log_destination = flow_log_info['LogDestination'][13:]
        flow_logs_table_name = "flow_logs"
        flow_logs_path = f"s3://{log_destination}/AWSLogs"
        flow_logs_type = flow_log_info['DestinationOptions']['FileFormat']
    

{'CreationTime': datetime.datetime(2022, 11, 3, 18, 46, 20, 172000, tzinfo=tzlocal()), 'DeliverLogsStatus': 'SUCCESS', 'FlowLogId': 'fl-07b3248185504499d', 'FlowLogStatus': 'ACTIVE', 'ResourceId': 'vpc-bf567fd8', 'TrafficType': 'ALL', 'LogDestinationType': 's3', 'LogDestination': 'arn:aws:s3:::flowlogs-bucket-cfncloudtraillogsbucket-1be4gsnc376f0', 'LogFormat': '${version} ${account-id} ${interface-id} ${srcaddr} ${dstaddr} ${srcport} ${dstport} ${protocol} ${packets} ${bytes} ${start} ${end} ${action} ${log-status}', 'Tags': [], 'MaxAggregationInterval': 600, 'DestinationOptions': {'FileFormat': 'parquet', 'HiveCompatiblePartitions': False, 'PerHourPartition': False}}


In [28]:
from jupyterirtools import cloudformation
import importlib
import boto3
importlib.reload(cloudformation)


org_id = desc_client['Organization']['Id']

athena_cloudtrail_stack = cloudformation.deploy(stack_name="athena-cloudtrail", 
                      role="Jupyter-IR-AdministratorAccess", 
                      account=os.environ['LOGGING_ACCOUNT'], 
                      template="../cfn-templates/athena-cloudtrail.yaml",
                      parameters={
                          "DatabaseName": 'jupyter-notebooks',
                          "RegionList": ','.join(regions),
                          "CloudTrailLogsBucketPath": cloudtrail_logs_location,
                          "CloudTrailTableName": 'cloudtrail_logs',
                          "FlowLogsBucketPath": flow_logs_path,
                          "FlowLogsTableName": flow_logs_table_name,
                          "FlowLogsFormat": flow_logs_type
                      })

athena_cloudtrail_stack

aws cloudformation deploy --stack-name=athena-cloudtrail --template-file ../cfn-templates/athena-cloudtrail.yaml --capabilities CAPABILITY_IAM --profile Jupyter-IR-AdministratorAccess-251344881676 --parameter-overrides  DatabaseName=jupyter-notebooks RegionList=eu-north-1,ap-south-1,eu-west-3,eu-west-2,eu-west-1,ap-northeast-3,ap-northeast-2,ap-northeast-1,sa-east-1,ca-central-1,ap-southeast-1,ap-southeast-2,eu-central-1,us-east-1,us-east-2,us-west-1,us-west-2 CloudTrailLogsBucketPath=s3://organization-logging-buck-cfncloudtraillogsbucket-3jqplzdobirg/AWSLogs/o-krcf45heml CloudTrailTableName=cloudtrail_logs FlowLogsBucketPath=s3://flowlogs-bucket-cfncloudtraillogsbucket-1be4gsnc376f0/AWSLogs FlowLogsTableName=flow_logs FlowLogsFormat=parquet

Waiting for changeset to be created..

No changes to deploy. Stack athena-cloudtrail is up to date


{'Name': 'athena-cloudtrail', 'Outputs': {}, 'Status': 'UPDATE_COMPLETE'}